# Challenge
- 1. Data uploading
- 2. Plotting maps
- 3. Creating models

## 1. Data loading

In [1]:
import pandas as pd
import pydeck as pdk

df = pd.read_csv('data_squares.csv')

In [2]:
df.head()
#df.describe()

,Unnamed: 0,square_id,center_lat,center_lng,average_hh_ha,average_pers_ha,average_pt_class,average_pt_dis,average_station_dis,average_noise_street,average_number_of_supermarket,average_popularity_percent_normal,average_number_of_migros,average_popularity_percent_normal_migros
0,0,sq1-1,47.360332,8.430295,0.000000,0.000000,3.956522,562.030075,8531.165414,52.398496,0,NaN,0,NaN
1,1,sq1-2,47.360081,8.434614,1.446809,3.063830,2.893617,372.180851,8220.904255,52.712766,0,NaN,0,NaN
2,2,sq1-3,47.361836,8.438194,0.000000,0.000000,3.500000,486.250000,7911.785714,66.071429,0,NaN,0,NaN
3,3,sq1-4,47.359902,8.445250,8.000000,19.076923,3.780220,158.021978,7450.054945,55.714286,0,NaN,0,NaN
4,4,sq1-5,47.359770,8.449631,3.478261,8.869565,3.000000,85.869565,7136.086957,62.804348,0,NaN,0,NaN


In [3]:
#Reversing pt_class data to match order with size of categorical value
df['average_pt_class'] = 5 - df['average_pt_class']

## 2. Plotting maps
- 2.1 Create 2 layers for all components (scatter plot and heatmap layer)
- 2.2 Define viewpoint
- 2.3 Choose the plots and print

#### 2.1 Create 2 layers for all components (scatter plot and heatmap layer)

In [4]:
#Scatter Layer
layer_scatter=[]
for elements in df:
    layer_scatter.append(pdk.Layer(
        "ScatterplotLayer",
        df,
        opacity=0.2,
        get_position=["center_lng", "center_lat"],
        radius_scale=5,
        radius_min_pixels=0,
        radius_max_pixels=20,
        line_width_min_pixels=1,
        get_radius=elements,
        get_fill_color=[10, 200, 150],
        get_line_color=[0, 0, 0]
        ))

In [5]:
#Heatmap Layer
layer_heatmap=[]
for elements in df:
    layer_heatmap.append(pdk.Layer(
    "HeatmapLayer",
    df,
    opacity=0.9,
    get_position=["center_lng", "center_lat"],
    aggregation=pdk.types.String("MEAN"),
    threshold=1,
    get_weight=elements,
    pickable=True,
    ))

In [6]:
#More layering examples
layer_sup = pdk.Layer(
        "ScatterplotLayer",
        df,
        opacity=0.2,
        get_position=["center_lng", "center_lat"],
        radius_scale=30,
        radius_min_pixels=0,
        radius_max_pixels=300,
        line_width_min_pixels=1,
        get_radius='average_number_of_supermarket',
        get_fill_color=[10, 200, 150],
        get_line_color=[0, 0, 0]
        )

layer_migro = pdk.Layer(
        "ScatterplotLayer",
        df,
        opacity=0.2,
        get_position=["center_lng", "center_lat"],
        radius_scale=30,
        radius_min_pixels=0,
        radius_max_pixels=300,
        line_width_min_pixels=1,
        get_radius='average_number_of_migros',
        get_fill_color=[200, 50, 50],
        get_line_color=[0, 0, 0]
        )

#### 2.2 Define viewpoint

In [7]:
view_state = pdk.ViewState(
    longitude= 8.522128,
    latitude=47.395171,
    zoom=11.5,
    min_zoom=5,
    max_zoom=20)

#### 2.3 Choose the plots and print

In [8]:
#Define deck by selecting the layer and a name for the html file
r = pdk.Deck(layers=[layer_sup,layer_migro],
             initial_view_state=view_state,
             map_style ='light')

r.to_html('zurich-scatter-nsup.html')

## 3. Creating models
- 3.1 Create normalized database (using normalization min-max)
- 3.2 Create hypersimplistic model with multiplications and division
- 3.3 Create correlation adjusted model

#### 3.1 Create normalized database (using normalization min-max) 

In [55]:
#I am using a min-max scaling
#Source: https://sebastianraschka.com/Articles/2014_about_feature_scaling.html#standardizing-and-normalizing---how-it-can-be-done-using-scikit-learn
import copy
import numpy as np
df_dummy = df.copy()
df_pre_norm = df_dummy.iloc[:,4:].reset_index()
#df_norm = (df_pre_norm-df_pre_norm.min())/(df_pre_norm.max()-df_pre_norm.min())
df_norm = (df_pre_norm-df_pre_norm.mean())/(df_pre_norm.std())
df_norm = df_dummy.iloc[:,0:4].join(df_norm)
#df_norm.head()

,Unnamed: 0,square_id,center_lat,center_lng,index,average_hh_ha,average_pers_ha,average_pt_class,average_pt_dis,average_station_dis,average_noise_street,average_number_of_supermarket,average_popularity_percent_normal,average_number_of_migros,average_popularity_percent_normal_migros
0,0,sq1-1,47.360332,8.430295,-1.727375,-0.928319,-0.980478,-1.656045,1.580142,1.910337,-0.033982,-0.430145,NaN,-0.294884,NaN
1,1,sq1-2,47.360081,8.434614,-1.721138,-0.859814,-0.906677,-0.601166,0.571226,1.764492,0.005664,-0.430145,NaN,-0.294884,NaN
2,2,sq1-3,47.361836,8.438194,-1.714902,-0.928319,-0.980478,-1.202970,1.177424,1.619184,1.690905,-0.430145,NaN,-0.294884,NaN
3,3,sq1-4,47.359902,8.445250,-1.708666,-0.549527,-0.520958,-1.481074,-0.566879,1.402138,0.384316,-0.430145,NaN,-0.294884,NaN
4,4,sq1-5,47.359770,8.449631,-1.702430,-0.763627,-0.766830,-0.706746,-0.950319,1.254550,1.278752,-0.430145,NaN,-0.294884,NaN


#### 3.2 Create hypersimplistic model with multiplications and division

In [61]:
#Here the pt_class is adequately sorted (highest value corresponds to the highest quality)
df_num_supermarkt = df_dummy['average_number_of_supermarket']
df_num_migros = df_dummy['average_number_of_migros']
df_num_supermarkt.replace(0,0.5)
df_num_migros.replace(0,0.5)

#df_norm['simple_result'] = (df_norm['average_pers_ha']*df_norm['average_pt_class']/df_num_supermarkt)*(df_num_migros/df_num_supermarkt)
df_norm['simple_result'] = (df_norm['average_pers_ha']*df_norm['average_pt_class'])*(df_num_supermarkt-df_num_migros)/df_num_supermarkt
df_norm.describe()

,Unnamed: 0,center_lat,center_lng,index,average_hh_ha,average_pers_ha,average_pt_class,average_pt_dis,average_station_dis,average_noise_street,average_number_of_supermarket,average_popularity_percent_normal,average_number_of_migros,average_popularity_percent_normal_migros,simple_result
count,555.000000,521.000000,521.000000,555.000000,5.210000e+02,5.210000e+02,5.080000e+02,5.210000e+02,5.210000e+02,5.210000e+02,5.550000e+02,1.220000e+02,555.000000,5.000000e+01,133.000000
mean,277.000000,47.395219,8.519979,0.000000,3.409514e-17,-9.546639e-17,-2.098059e-16,-1.363806e-17,3.545895e-16,8.864737e-16,6.401286e-18,-1.365028e-16,0.000000,-2.398082e-16,0.593855
std,160.358972,0.021286,0.052523,1.000000,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000,1.000000e+00,0.992558
min,0.000000,47.358900,8.428646,-1.727375,-9.283185e-01,-9.804783e-01,-1.699195e+00,-9.848332e-01,-1.977164e+00,-3.490398e+00,-4.301446e-01,-3.238208e+00,-0.294884,-2.554104e+00,-0.906580
25%,138.500000,47.375201,8.475183,-0.863687,-8.375123e-01,-8.606152e-01,-8.105020e-01,-6.189014e-01,-7.174626e-01,-5.370787e-01,-4.301446e-01,-5.529400e-01,-0.294884,-3.622573e-01,0.000000
50%,277.000000,47.395079,8.519395,0.000000,-3.548067e-01,-3.583010e-01,2.083563e-01,-4.359580e-01,-1.391968e-01,1.502267e-01,-4.301446e-01,5.532730e-02,-0.294884,2.176526e-01,0.318196
75%,415.500000,47.414825,8.564184,0.863687,5.720158e-01,6.500206e-01,8.935998e-01,2.545745e-01,6.232814e-01,6.454420e-01,-4.301446e-01,6.535155e-01,-0.294884,6.724301e-01,0.990367
max,554.000000,47.430959,8.609374,1.727375,4.760565e+00,4.400994e+00,1.278152e+00,4.455345e+00,2.652550e+00,2.465049e+00,9.353883e+00,2.435426e+00,7.258678,2.027086e+00,4.669923


In [92]:
layer_simple_model = pdk.Layer(
    "HeatmapLayer",
    df_norm,
    opacity=0.9,
    get_position=["center_lng", "center_lat"],
    aggregation=pdk.types.String("MEAN"),
    threshold=1,
    get_weight='simple_result',
    pickable=True,
    )

r = pdk.Deck(layers=[layer_simple_model],
             initial_view_state=view_state,
             map_style ='light')

r.to_html('zurich-simple-model.html')

#### 3.3 Create correlation adjusted model
- Calculate correlation coefficients metrics
- 

In [76]:
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
#Manipulate the existing number of supermarket column
df_dummy1 = df_norm.copy()
df_dummy1 = df_dummy1[['average_pers_ha','average_number_of_supermarket','average_pt_class','average_popularity_percent_normal']]
df_dummy2 = df_dummy1[['average_pers_ha','average_number_of_supermarket','average_pt_class']].dropna()
df_dummy3 = df_dummy1.dropna()
#df_num_supermarkt = df_dummy2['average_number_of_supermarket']
#df_ppl = df_dummy2['average_pers_ha'].dropna()

model = sm.GLM(df_dummy2['average_number_of_supermarket'],df_dummy2['average_pers_ha'],family=sm.families.Poisson())
gamma_results = model.fit()
print(gamma_results.summary())
print(gamma_results)
#t = np.arange(0, 20, 0.1)
#d = np.exp(-5)*np.power(5, t)/factorial(t)

                       Generalized Linear Model Regression Results                       
Dep. Variable:     average_number_of_supermarket   No. Observations:                  508
Model:                                       GLM   Df Residuals:                      507
Model Family:                            Poisson   Df Model:                            0
Link Function:                               log   Scale:                          1.0000
Method:                                     IRLS   Log-Likelihood:                -718.99
Date:                           Fri, 10 Sep 2021   Deviance:                       12804.
Time:                                   13:21:11   Pearson chi2:                     971.
No. Iterations:                                6                                         
Covariance Type:                       nonrobust                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
----------------

In [72]:
model = sm.GLM(df_dummy2['average_number_of_supermarket'],df_dummy2['average_pt_class'],family=sm.families.Poisson())
gamma_results = model.fit()
print(gamma_results.summary())
print(gamma_results)

                       Generalized Linear Model Regression Results                       
Dep. Variable:     average_number_of_supermarket   No. Observations:                  508
Model:                                       GLM   Df Residuals:                      507
Model Family:                            Poisson   Df Model:                            0
Link Function:                               log   Scale:                          1.0000
Method:                                     IRLS   Log-Likelihood:                -707.71
Date:                           Fri, 10 Sep 2021   Deviance:                       12793.
Time:                                   12:51:47   Pearson chi2:                     919.
No. Iterations:                                6                                         
Covariance Type:                       nonrobust                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
---------------

In [80]:
model = sm.GLM(df_dummy3['average_number_of_supermarket'],df_dummy3['average_popularity_percent_normal'],family=sm.families.Poisson())
gamma_results = model.fit()
print(gamma_results.summary())
print(gamma_results)

                       Generalized Linear Model Regression Results                       
Dep. Variable:     average_number_of_supermarket   No. Observations:                  122
Model:                                       GLM   Df Residuals:                      121
Model Family:                            Poisson   Df Model:                            0
Link Function:                               log   Scale:                          1.0000
Method:                                     IRLS   Log-Likelihood:                -192.75
Date:                           Fri, 10 Sep 2021   Deviance:                       131.66
Time:                                   13:29:22   Pearson chi2:                     231.
No. Iterations:                                4                                         
Covariance Type:                       nonrobust                                         
                                        coef    std err          z      P>|z|      [0.025      0.975

#### Create a coefficient adjusted model
- Even assignment to all parameters (33-33-33)
- Heavier weights to transport and occupancy rate (40-40-20)
- Heavier occupancy (10-10-80)

In [84]:
#Here the pt_class is adequately sorted (highest value corresponds to the highest quality)
df_num_supermarkt = df_dummy['average_number_of_supermarket']
df_num_migros = df_dummy['average_number_of_migros']
df_num_supermarkt.replace(0,0.5)
df_num_migros.replace(0,0.5)
df_norm['average_popularity_percent_normal'] = df_norm['average_popularity_percent_normal'].fillna(0)

#df_norm['simple_result'] = (df_norm['average_pers_ha']*df_norm['average_pt_class']/df_num_supermarkt)*(df_num_migros/df_num_supermarkt)
df_norm['coeficient_adjusted_result'] = 0.2830*0.333*df_norm['average_pers_ha']+0.3935*0.3333*df_norm['average_pt_class']-0.1371*0.3333*df_norm['average_popularity_percent_normal']
df_norm['coeficient_adjusted_result'] = df_norm['coeficient_adjusted_result']/df_num_supermarkt

In [96]:
layer_simple_model = pdk.Layer(
    "HeatmapLayer",
    df_norm,
    opacity=0.9,
    get_position=["center_lng", "center_lat"],
    aggregation=pdk.types.String("MEAN"),
    threshold=1,
    get_weight='coeficient_adjusted_result',
    pickable=True,
    )

r = pdk.Deck(layers=[layer_simple_model],
             initial_view_state=view_state,
             map_style ='light')

r.to_html('zurich-coeficient_adj-model.html')

In [97]:
df_norm['coeficient_adjusted_result_1'] = 0.2830*0.4*df_norm['average_pers_ha']+0.3935*0.2*df_norm['average_pt_class']-0.1371*0.4*df_norm['average_popularity_percent_normal']
df_norm['coeficient_adjusted_result_1'] = df_norm['coeficient_adjusted_result_1']/df_num_supermarkt
layer_simple_model = pdk.Layer(
    "HeatmapLayer",
    df_norm,
    opacity=0.9,
    get_position=["center_lng", "center_lat"],
    aggregation=pdk.types.String("MEAN"),
    threshold=1,
    get_weight='coeficient_adjusted_result_1',
    pickable=True,
    )

r = pdk.Deck(layers=[layer_simple_model],
             initial_view_state=view_state,
             map_style ='light')

r.to_html('zurich-coeficient_adj-model_1.html')

In [98]:
df_norm['coeficient_adjusted_result_2'] = 0.2830*0.2*df_norm['average_pers_ha']+0.3935*0.2*df_norm['average_pt_class']-0.1371*0.6*df_norm['average_popularity_percent_normal']
df_norm['coeficient_adjusted_result_2'] = df_norm['coeficient_adjusted_result_2']/df_num_supermarkt
layer_simple_model = pdk.Layer(
    "HeatmapLayer",
    df_norm,
    opacity=0.9,
    get_position=["center_lng", "center_lat"],
    aggregation=pdk.types.String("MEAN"),
    threshold=1,
    get_weight='coeficient_adjusted_result_2',
    pickable=True,
    )

r = pdk.Deck(layers=[layer_simple_model],
             initial_view_state=view_state,
             map_style ='light')

r.to_html('zurich-coeficient_adj-model_2.html')